<a href="https://colab.research.google.com/github/Raynould-Joseph/Neural-Network-and-Deep-Learning/blob/main/SimpleRNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Implement the given tasks in sequence
a. Generating sample dataset/use any open data set
b. Preprocess data (reshaping)
c. Building a SimpleRNN model
d. Print or plot the prediction results

2. Rebuild a GRU or LSTM model from a textbook or online sources for text
classification. Interpret your observations during the lab evaluation. [optional
question for additional learning]